In [2]:
import requests 
import yaml
import json
import datetime as dt
import time
import os
import random
from azure.storage.blob import BlobServiceClient, BlobType
from email_validator import validate_email, EmailNotValidError
from funcs.utils import log_azure, request, load_json, clean_field_text, has_valid_email, load_config
from funcs.funcs import combine_qa_keys, get_sm_survey_responses, process_sm_responses

with open("api-key.yaml", "r") as file:
    data = yaml.full_load(file)

# SurveyMonkey Survey
SM_DATA = data['sm']['real']
# CareerOneStop Survey 
COS_DATA = data['cos']


---

##### **Main Functions** 

* `get_qa_key()` - GET (or load cached copy) of question/answer key from SM or COS 

<br>

* `combine_qa_keys()` - Combine the SM and COS question/answer keys into one combined key/translation map between the APIs
    - Generates a refreshed map if a change is detected in the SM survey or the COS survey. 
        - The COS survey should not change at all. 
        - The survey monkey questions which match to the COS survey questions should not change (they are intended to just be a port).

<br>

* `get_sm_responses()` - GET SM survey responses 

<br>

* `process_sm_responses()` - filter and process new SM survey responses from get_sm_responses()
    - Checks against DB for already processed responses 
    - Checks for unexpected question ids vs. the combined Q/A key.
        - Attempts to refresh the combined Q/A key if any unexpected question ids are found.
    - Adds matching information from the combined Q/A key to the survey responses   
    - Loads new responses into database (into 'processing' table) until they are finished

<br>

* `post_cos()` 
    - Creates COS JSON request objects from each SM survey response object 
        - If a SM survey response is missing an answer for a skills-survey question, it fills the corresponding question in the COS object with an answer of "Beginner"
    -  POSTS each request object to the COS Skills Matcher API 
    - Stores the COS response alongside the original SM survey response, updating the database 
    
<br>

* `compose_email()` - Compose an email from a COS response object.

<br>

* `send_email()` - Send email if respondent provided valid email address. 

<br>

* `export_csv()` - Export data from database in csv/excel format for Social Contract analysis purposes      

In [3]:
sm_survey_responses = get_sm_survey_responses()

INFO: GET {'https://api.surveymonkey.com/v3/surveys/513506444/responses/bulk'} -- {200} -- 2.50 -- {datetime.datetime(2023, 10, 11, 12, 24, 32, 207769)}


In [4]:
sm_survey_responses

{'data': [{'id': '118440150887',
   'recipient_id': '',
   'collection_mode': 'default',
   'response_status': 'completed',
   'custom_value': '',
   'first_name': '',
   'last_name': '',
   'email_address': '',
   'ip_address': '73.165.183.94',
   'logic_path': {},
   'metadata': {'contact': {}},
   'page_path': [],
   'collector_id': '452261040',
   'survey_id': '513506444',
   'custom_variables': {},
   'edit_url': 'https://www.surveymonkey.com/r/?sm=MSXCvuilN9Vf0ZobVXFjPswyoNLgo_2FfEjRoZb0vM7grlb8CTskkkTNj66m1GwMlq',
   'analyze_url': 'https://www.surveymonkey.com/analyze/browse/8xd9D8KA37YeLpeTvx8j2O9lfNbj2kqrYZtRcHvmtdM_3D?respondent_id=118440150887',
   'total_time': 1043,
   'date_modified': '2023-10-10T23:03:01+00:00',
   'date_created': '2023-10-10T22:45:38+00:00',
   'href': 'https://api.surveymonkey.com/v3/surveys/513506444/responses/118440150887',
   'pages': [{'id': '43673968',
     'questions': [{'id': '144588883',
       'answers': [{'choice_id': '1070603277'}]},
      